In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import HTML
%matplotlib inline
SanityCheck = True
FrameCount = 0

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        
        f, (ax1) = plt.subplots(1, 1, figsize=(24, 9))
        f.tight_layout()
        ax1.imshow(img)

ret, mat_undistort, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [ ]:
if SanityCheck:
    fname = images[9] #Choose any image
    img = cv2.imread(fname)
    undist = cv2.undistort(img, mat_undistort, dist, None, mat_undistort)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undist)
    ax2.set_title('Undistorted Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
def perspective_transform(persp_M, pts):#persp_M is 3x3 matrix and pts is Nx2 array of points
    pts_aff = pts.transpose()
    pts_aff = np.vstack((pts_aff, np.ones((1,pts_aff.shape[1]))))
    trans_aff = np.matmul(persp_M,pts_aff)
    trans = np.divide(trans_aff[:-1,:], trans_aff[-1,:])
    return(trans)

In [ ]:
src = np.float32([[210,719], [1105, 719], [685, 450], [595,450]])
dst = np.float32([[350, 719], [950, 719], [950, 0], [350, 0]])
#src = np.float32([[210,719], [1105, 719], [765, 500], [525,500]])
#dst = np.float32([[350, 719], [950, 719], [950, 0], [350, 0]])

# Given src and dst points, calculate the perspective transform matrix
mat_persp_trans = cv2.getPerspectiveTransform(src, dst)
mat_inv_persp_trans = np.linalg.inv(mat_persp_trans)

#Printing the values desired and achieved transformation of the source points
transp = perspective_transform(mat_persp_trans, src); transp = transp.transpose().astype(np.int32);
print('The desired destination points are :')
print(dst.astype(np.int32))
print('\nThe transformation matrix maps them to:')
print(transp);
print('The transformation seems to be fine.')

In [ ]:
if SanityCheck:
    img = cv2.imread('./test_images/straight_lines1.jpg')
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img_size = (img.shape[1], img.shape[0])
    undist = cv2.undistort(img, mat_undistort, dist, None, mat_undistort)
    warped = cv2.warpPerspective(undist, mat_persp_trans, (img.shape[1], img.shape[0]))
    
    #Plot the images
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undist)
    ax2.add_patch(patches.Polygon(src, fill=False, edgecolor='red', linewidth=3.5))
    ax2.set_title('Undistorted Image', fontsize=50)
    ax3.imshow(warped)
    ax3.add_patch(patches.Polygon(transp, fill=False, edgecolor='red', linewidth=3.5))
    ax3.set_title('warped', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)